In [1]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)

In [2]:
# LiH: experimental 1.595A, calculated 1.618A 

molecule = Molecule(
    geometry=[["Li", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 1.595]]], charge=0, multiplicity=1
)
driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
)

In [3]:
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper

In [6]:
es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()
print(second_q_op[0])
print("num particles: ", es_problem.num_particles)
print("num spin orbitals: ", es_problem.num_spin_orbitals)

Fermionic Operator
register length=12, number terms=630
  (0.0011682892995386579+0j) * ( +_0 +_1 -_2 -_5 )
+ (-0.005362879199196197+0j) * ( +_0 -_1 +_2 -_5 )
+ (0.004194589899657527+0j) * ( +_0 -_1 -_2 +_5 )
+ (0.013396834058767583+0j) * ( +_0 -_1 +_6  ...
num particles:  (2, 2)
num spin orbitals:  12


In [8]:
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
qubit_op = qubit_converter.convert(second_q_op[0])
print(type(qubit_op))
print(qubit_op)

<class 'qiskit.opflow.primitive_ops.pauli_sum_op.PauliSumOp'>
-5.12960333830415 * IIIIIIIIIIII
- 0.3857012469009301 * ZIIIIIIIIIII
- 0.23039162013944758 * IZIIIIIIIIII
+ 0.062120196937157104 * ZZIIIIIIIIII
- 0.23039162013944756 * IIZIIIIIIIII
+ 0.062120196937157084 * ZIZIIIIIIIII
+ 0.06558452315458407 * IZZIIIIIIIII
- 0.19806776139549948 * IIIZIIIIIIII
+ 0.05375778410543414 * ZIIZIIIIIIII
+ 0.060181494845631865 * IZIZIIIIIIII
+ 0.060181494845631865 * IIZZIIIIIIII
- 0.003981664807069578 * XZIXIIIIIIII
- 0.003981664807069588 * XIZXIIIIIIII
+ 0.01467800268862425 * XZZXIIIIIIII
- 0.003981664807069578 * YZIYIIIIIIII
- 0.003981664807069588 * YIZYIIIIIIII
+ 0.01467800268862425 * YZZYIIIIIIII
- 0.11830062685800538 * IIIIZIIIIIII
+ 0.0825424080645461 * ZIIIZIIIIIII
+ 0.06174201105109741 * IZIIZIIIIIII
+ 0.0617420110510974 * IIZIZIIIIIII
+ 0.05268477696985793 * IIIZZIIIIIII
+ 0.004199945724056172 * XZZXZIIIIIII
+ 0.004199945724056172 * YZZYZIIIIIII
+ 1.0066954765126432 * IIIIIZIIIIII
+ 0.0883127

In [9]:
qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)
qubit_op = qubit_converter.convert(second_q_op[0], num_particles=es_problem.num_particles)
print(type(qubit_op))
print(qubit_op)

<class 'qiskit.opflow.primitive_ops.tapered_pauli_sum_op.TaperedPauliSumOp'>
-5.129603338304149 * IIIIIIIIII
- 0.38570124690092994 * ZIIIIIIIII
+ 0.06212019693715708 * IZIIIIIIII
- 0.2303916201394475 * ZZIIIIIIII
+ 0.06558452315458405 * ZIZIIIIIII
- 0.23039162013944747 * IZZIIIIIII
+ 0.06212019693715707 * ZZZIIIIIII
+ 0.08847831679250717 * IIIZIIIIII
+ 0.060181494845631844 * IZIZIIIIII
- 0.1980677613954994 * IIZZIIIIII
+ 0.053757784105434116 * ZIZZIIIIII
+ 0.060181494845631844 * ZZZZIIIIII
+ 1.0066954765126428 * IIIIZIIIII
+ 0.08831277199612228 * ZIIIZIIIII
+ (0.09662524704093835+6.938893903907228e-18j) * ZZIIZIIIII
+ 0.05268477696985791 * IIZIZIIIII
+ (0.0966252470409383-6.938893903907228e-18j) * IZZIZIIIII
- 0.11830062685800533 * IIIZZIIIII
+ 0.08254240806454607 * ZIIZZIIIII
+ 0.0617420110510974 * ZZIZZIIIII
+ (0.09349956480100238-6.938893903907228e-18j) * IIZZZIIIII
+ 0.061742011051097376 * IZZZZIIIII
+ (-0.38570124690092994+2.7755575615628914e-17j) * IIIIIZIIII
+ 0.1134896282877146

In [10]:
# Classical Solver
from qiskit.algorithms import NumPyMinimumEigensolver
numpy_solver = NumPyMinimumEigensolver()

In [11]:
# Initialize VQE Solver
from qiskit.providers.aer import StatevectorSimulator
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import VQEUCCFactory

quantum_instance = QuantumInstance(backend=Aer.get_backend("aer_simulator_statevector"))
vqe_solver = VQEUCCFactory(quantum_instance)

In [12]:
from qiskit_nature.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(qubit_converter, vqe_solver)
res = calc.solve(es_problem)

print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.877708737437
  - computed part:      -8.877708737437
~ Nuclear repulsion energy (Hartree): 0.995317638094
> Total ground state energy (Hartree): -7.882391099343
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  3.01411317]
 
  0: 
  * Electronic dipole moment (a.u.): [0.00000052  -0.00000043  4.83239391]
    - computed part:      [0.00000052  -0.00000043  4.83239391]
  > Dipole moment (a.u.): [-0.00000052  0.00000043  -1.81828074]  Total: 1.81828074
                 (debye): [-0.00000132  0.0000011  -4.62160821]  Total: 4.62160821
 


In [13]:
calc = GroundStateEigensolver(qubit_converter, numpy_solver)
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.877719570384
  - computed part:      -8.877719570384
~ Nuclear repulsion energy (Hartree): 0.995317638094
> Total ground state energy (Hartree): -7.88240193229
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  3.01411317]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  4.83182882]
    - computed part:      [0.0  0.0  4.83182882]
  > Dipole moment (a.u.): [0.0  0.0  -1.81771565]  Total: 1.81771565
                 (debye): [0.0  0.0  -4.62017191]  Total: 4.62017191
 
